In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
import warnings
warnings.filterwarnings('ignore')

In [138]:
credentials = service_account.Credentials.from_service_account_file(
    '/Users/peter/Documents/positive-water-229419-e00e7547dca7.json')
project_id = 'positive-water-229419'

query = '''
#StandardSQL
SELECT t3.fullVisitorId, date, min_date, visitNumber, visitStartTime, channelGrouping, geoNetwork.continent, geoNetwork.country,
device.operatingSystem, device.browser, totals.pageviews
FROM
(SELECT fullVisitorId, min(date) as min_date
FROM
`bigquery-public-data.google_analytics_sample.ga_sessions_*`
GROUP BY fullVisitorId) t12
INNER JOIN
`bigquery-public-data.google_analytics_sample.ga_sessions_*` t3
ON t12.fullVisitorId = t3.fullVisitorId 
WHERE min_date BETWEEN '20170101' AND '20170106'
'''
df = pd.read_gbq(query, project_id=project_id, credentials=credentials, dialect = 'standard')


In [149]:
df.head(10)

,fullVisitorId,date,min_date,visitNumber,visitStartTime,channelGrouping,continent,country,operatingSystem,browser,pageviews,timestamp,timestamp_min
0,1582358441388888900,20170308,20170105,5,1488986931,Organic Search,Europe,Netherlands,Windows,Chrome,1,1488920400,1483563600
1,3399585678206234119,20170308,20170105,2,1489027616,Organic Search,Americas,United States,iOS,Safari,1,1488920400,1483563600
2,5841100789157331084,20170308,20170102,2,1489011425,Direct,Americas,United States,iOS,Chrome,1,1488920400,1483304400
3,2065005233687444469,20170308,20170102,2,1489035491,Organic Search,Americas,United States,Android,Chrome,1,1488920400,1483304400
4,6975616481186052982,20170308,20170103,23,1488969410,Direct,Europe,United Kingdom,Windows,Chrome,2,1488920400,1483390800
5,0290663072707523580,20170308,20170105,11,1489017566,Organic Search,Americas,United States,Macintosh,Firefox,14,1488920400,1483563600
6,0290663072707523580,20170308,20170105,10,1489010099,Organic Search,Americas,United States,Macintosh,Firefox,14,1488920400,1483563600
7,6753461169524062240,20170308,20170106,2,1488969314,Social,Asia,Iraq,iOS,Safari,1,1488920400,1483650000
8,0727684956809986344,20170118,20170106,3,1484727849,Organic Search,Asia,Japan,Windows,Chrome,1,1484686800,1483650000
9,724337952100421725,20170118,20170104,2,1484770014,Organic Search,Americas,United States,Windows,Chrome,1,1484686800,1483477200


In [140]:
df.size

127556

In [153]:
# Метрики для наблюдения - количество просмотренных страниц и количество визитов
# Интервал когорты - неделя
# Период - 4 недели
# Тип когорты - дата первого визита

In [141]:
df.isna().sum()

fullVisitorId      0
date               0
min_date           0
visitNumber        0
visitStartTime     0
channelGrouping    0
continent          0
country            0
operatingSystem    0
browser            0
pageviews          0
dtype: int64

In [142]:
def preparation(df):
    df.pageviews = df.pageviews.fillna(0).astype(int)
    df['timestamp'] = df[['date']].apply(lambda x: int(datetime.strptime(str(x[0]), '%Y%m%d')\
                                                            .strftime("%s")), axis=1)
    df['timestamp_min'] = df[['min_date']].apply(lambda x: int(datetime.strptime(str(x[0]), '%Y%m%d')\
                                                            .strftime("%s")), axis=1)
    return df   

In [143]:
df = preparation(df)

In [144]:
df.head()

,fullVisitorId,date,min_date,visitNumber,visitStartTime,channelGrouping,continent,country,operatingSystem,browser,pageviews,timestamp,timestamp_min
0,1582358441388888900,20170308,20170105,5,1488986931,Organic Search,Europe,Netherlands,Windows,Chrome,1,1488920400,1483563600
1,3399585678206234119,20170308,20170105,2,1489027616,Organic Search,Americas,United States,iOS,Safari,1,1488920400,1483563600
2,5841100789157331084,20170308,20170102,2,1489011425,Direct,Americas,United States,iOS,Chrome,1,1488920400,1483304400
3,2065005233687444469,20170308,20170102,2,1489035491,Organic Search,Americas,United States,Android,Chrome,1,1488920400,1483304400
4,6975616481186052982,20170308,20170103,23,1488969410,Direct,Europe,United Kingdom,Windows,Chrome,2,1488920400,1483390800


In [146]:
def week4_cohort_by_first_day(dftmp, cohort_day, weeks=4):
    cohort_day = datetime.strptime(str(cohort_day), '%Y%m%d' ).timestamp()
    dftmp = dftmp[dftmp['timestamp_min'] == cohort_day]
    dictionary = {}
    tmp_week_timestamp = 7*24*60*60
    for i in range(1,weeks+1):
        dictionary['week_'+str(i)] = tmp_week_timestamp*i 

    dftmp['week'] = df.apply(lambda x: 'week_1' if x.timestamp <= (x.timestamp_min + dictionary['week_1']) 
        else 'week_2' if (x.timestamp > (x.timestamp_min + dictionary['week_1'])) and 
                      (x.timestamp <= (x.timestamp_min + dictionary['week_2'])) 
        else 'week_3' if (x.timestamp > (x.timestamp_min + dictionary['week_2'])) and 
                      (x.timestamp <= (x.timestamp_min + dictionary['week_3'])) 
        else 'week_4' if (x.timestamp > (x.timestamp_min + dictionary['week_3'])) and 
                      (x.timestamp <= (x.timestamp_min + dictionary['week_4'])) 
        else 'other_weeks', axis=1)
    dftmp = dftmp.groupby(['week','week'])['week'].count().unstack().reset_index().fillna(0).iloc[:,2:]
    for key in dictionary:
        dftmp[key] = dftmp[key].sum()
    dftmp = dftmp.iloc[-1:,:]   
    return dftmp

In [147]:
def concatenation_cohorts_in_df(df, datelist):
    col=['week','week_1','week_2','week_3','week_4','date']
    zerodf = pd.DataFrame(columns=col)
    for i in datelist:
        temp = week4_cohort_by_first_day(df,str(i),weeks=4)
        temp['date'] = i
        zerodf = pd.concat([temp, zerodf])
    return zerodf

In [148]:
#список дат с первым посещением для формирования когорты
concatenation_cohorts_in_df(df,[20170101,20170102,20170103,20170104,20170105,20170106])

,date,week,week_1,week_2,week_3,week_4
4,20170106,NaN,1937.0,38.0,20.0,20.0
4,20170105,NaN,1884.0,42.0,38.0,20.0
4,20170104,NaN,2046.0,53.0,22.0,23.0
4,20170103,NaN,1948.0,41.0,42.0,32.0
4,20170102,NaN,1473.0,42.0,19.0,31.0
4,20170101,NaN,1200.0,19.0,19.0,13.0
